In [ ]:
'''
CLASSIFICATION INTO 3 CATEGORIES - LESS THAN 100k SAMPLES - NUMERIC
'''

In [56]:
import pandas as pd
from sqlalchemy import create_engine

sqlcon = create_engine('mssql+pyodbc://@' + 'GHOST-117\SQLEXPRESS' + '/' + 'MOHSIN' + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
# Now use SQL Query to bring in whatever - USE THIS QUERY TO BRING IN SPECIFIC DATA:
X_train1 = pd.read_sql_query("SELECT DISTINCT * FROM  [DRIVENDATA].[dbo].[watertable_X_train_PROCESSED]  ORDER BY ID", sqlcon)

y_train1 = pd.read_sql_query("SELECT DISTINCT * FROM  [DRIVENDATA].[dbo].[watertable_y_train_PROCESSED]  ORDER BY ID", sqlcon)

X_test1 = pd.read_sql_query("SELECT DISTINCT * FROM  [DRIVENDATA].[dbo].[watertable_X_test_PROCESSED]  ORDER BY ID", sqlcon)


In [57]:
X_train1.head()

,ID,WELL_AGE,POP_N,HEIGHT_N,quality_group,quantity_group,extraction_type_class2,management2,payment_type2,source2,waterpoint_type2,PERMIT_0,PERMIT_1
0,0,24,0.000,0.000000,milky,enough,handpump,vwc,unknown_pay,shallow well,hand pump,0,1
1,1,12,0.020,0.714079,good,enough,rope pump,vwc,never pay,shallow well,hand pump,1,0
2,2,24,0.000,0.000000,good,insufficient,motorpump,vwc,per bucket,machine dbh,communal standpipe multiple,0,1
3,3,21,0.025,0.591696,good,enough,gravity,water board,per bucket,spring,communal standpipe,0,1
4,4,24,0.000,0.000000,good,dry,handpump,vwc,unknown_pay,shallow well,hand pump,0,1


In [58]:
'''
dummy variables needed for:
[extraction_type_class2]
[management2]
[payment_type2]
[source2]
[waterpoint_type2]
'''

'\ndummy variables needed for:\n[extraction_type_class2]\n[management2]\n[payment_type2]\n[source2]\n[waterpoint_type2]\n'

In [59]:
dummies_ext = pd.get_dummies(X_train1.extraction_type_class2)
dummies_mgt = pd.get_dummies(X_train1.management2)
dummies_pay = pd.get_dummies(X_train1.payment_type2)
dummies_src = pd.get_dummies(X_train1.source2)
dummies_wtr = pd.get_dummies(X_train1.waterpoint_type2)

## try this:
dummies_qtg = pd.get_dummies(X_train1.quantity_group)
dummies_qlg = pd.get_dummies(X_train1.quality_group)

dummies_qtg.head()

,dry,enough,insufficient,seasonal,unknown
0,0,1,0,0,0
1,0,1,0,0,0
2,0,0,1,0,0
3,0,1,0,0,0
4,1,0,0,0,0


In [60]:
# Dummy Trap: drop all the 'other' columns
dummies_ext2 = dummies_ext.drop(['other_ext'], axis='columns')
dummies_mgt2 = dummies_mgt.drop(['other_mgt'], axis='columns')
dummies_pay2 = dummies_pay.drop(['other_pay'], axis='columns')
dummies_src2 = dummies_src.drop(['other_src'], axis='columns')
dummies_wtr2 = dummies_wtr.drop(['other_wtr'], axis='columns')

#
dummies_qtg2 = dummies_qtg.drop(['unknown'], axis='columns')
dummies_qlg2 = dummies_qlg.drop(['unknown'], axis='columns')


In [61]:
# Combine
X_train2 = pd.concat([X_train1,dummies_ext2], axis='columns')
X_train3 = pd.concat([X_train2,dummies_mgt2], axis='columns')
X_train4 = pd.concat([X_train3,dummies_pay2], axis='columns')
X_train5 = pd.concat([X_train4,dummies_src2], axis='columns')
X_train6 = pd.concat([X_train5,dummies_wtr2], axis='columns')

#
X_train7 = pd.concat([X_train6,dummies_qtg2], axis='columns')
X_train8 = pd.concat([X_train7,dummies_qlg2], axis='columns')

In [62]:
X_train8.head()

,ID,WELL_AGE,POP_N,HEIGHT_N,quality_group,quantity_group,extraction_type_class2,management2,payment_type2,source2,...,improved spring,dry,enough,insufficient,seasonal,colored,fluoride,good,milky,salty
0,0,24,0.000,0.000000,milky,enough,handpump,vwc,unknown_pay,shallow well,...,0,0,1,0,0,0,0,0,1,0
1,1,12,0.020,0.714079,good,enough,rope pump,vwc,never pay,shallow well,...,0,0,1,0,0,0,0,1,0,0
2,2,24,0.000,0.000000,good,insufficient,motorpump,vwc,per bucket,machine dbh,...,0,0,0,1,0,0,0,1,0,0
3,3,21,0.025,0.591696,good,enough,gravity,water board,per bucket,spring,...,0,0,1,0,0,0,0,1,0,0
4,4,24,0.000,0.000000,good,dry,handpump,vwc,unknown_pay,shallow well,...,0,1,0,0,0,0,0,1,0,0


In [63]:
# Drop varchar labels (for X and y)
X_train9 = X_train8.drop(['ID','extraction_type_class2','management2','payment_type2','source2','waterpoint_type2','quantity_group','quality_group'], axis='columns')
y_train2 = y_train1.drop(['ID'], axis='columns')

In [64]:
X_train9.head()

,WELL_AGE,POP_N,HEIGHT_N,PERMIT_0,PERMIT_1,gravity,handpump,motorpump,rope pump,submersible,...,improved spring,dry,enough,insufficient,seasonal,colored,fluoride,good,milky,salty
0,24,0.000,0.000000,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
1,12,0.020,0.714079,1,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
2,24,0.000,0.000000,0,1,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
3,21,0.025,0.591696,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
4,24,0.000,0.000000,0,1,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,0


In [65]:
'''
INITIAL MODEL SELECTION:

Cross Val checks:

Model -- Default Accuracy:


RandomForestClassifier -- 77%
HistGradientBoostingClassifier - 77% (processing intensive)
ExtraTreesClassifier - 77%

BaggingClassifier -- 76%
KNeighborsClassifier - 74%
GradientBoostingClassifier - 74%
DecisionTreeClassifier - 74%
AdaBoostClassifier - 72%
LogisticRegression -- 70%
SVC  -- 69% (took a long time!)
LinearSVC -- 62%
GaussianNB -- 22%
RandomForestRegressor - 47%
GradientBoostingRegressor - 37%

'''

'\nINITIAL MODEL SELECTION:\n\nCross Val checks:\n\nModel -- Default Accuracy:\n\n\nRandomForestClassifier -- 77%\nHistGradientBoostingClassifier - 77% (processing intensive)\nExtraTreesClassifier - 77%\n\nBaggingClassifier -- 76%\nKNeighborsClassifier - 74%\nGradientBoostingClassifier - 74%\nDecisionTreeClassifier - 74%\nAdaBoostClassifier - 72%\nLogisticRegression -- 70%\nSVC  -- 69% (took a long time!)\nLinearSVC -- 62%\nGaussianNB -- 22%\nRandomForestRegressor - 47%\nGradientBoostingRegressor - 37%\n\n'

In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier



from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier

In [53]:
########## try cross_val first, see what the processing time is like, before gridsearch

In [54]:
from sklearn.model_selection import cross_val_score

In [40]:
l_scores = cross_val_score(RandomForestClassifier(), X_train9, y_train2.status_group_2, cv = 3)
# accuracy
print("Accuracy: %0.2f (+/- %0.2f)" % (l_scores.mean(), l_scores.std() * 2))

Accuracy: 0.77 (+/- 0.01)


In [41]:
######### focus on randomforest:
l_scores = cross_val_score(RandomForestClassifier(n_estimators=400, max_features=None, max_depth=None, min_samples_leaf=5), X_train9, y_train2.status_group_2, cv = 3)
# accuracy
print("Accuracy: %0.2f (+/- %0.2f)" % (l_scores.mean(), l_scores.std() * 2))

Accuracy: 0.78 (+/- 0.00)


In [29]:
### Processing time is an issue... 

'''
HYPERTUNING 

PROCEEDING WITH ENSEMBLE METHODS

RandomForestClassifier -- (DEFAULT [n_estimators=100] ACCURACY: 77%)
ExtraTreesClassifier---  - 77%

 


---- with gridsearch:

RandomForestClassifier -- 
{n_estimators=40, max_features=None, max_depth=None, min_samples_leaf=5, 'min_samples_split': 3}  0.78%


ExtraTreesClassifier --
{'max_features': 'auto', 'n_estimators': 200} -- 77%

'''



"\nHYPERTUNING \n\nPROCEEDING WITH ENSEMBLE METHODS\n\nRandomForestClassifier -- (DEFAULT [n_estimators=100] ACCURACY: 77%)\nExtraTreesClassifier---  - 77%\n\n \n\n\n---- with gridsearch:\n\nRandomForestClassifier -- \n{'max_features': 'auto', 'n_estimators': 200} =  77.5% \n\n{'max_depth': None, 'min_samples_leaf': 5, 'min_samples_split': 3}\n\nExtraTreesClassifier --\n{'max_features': 'auto', 'n_estimators': 200} -- 77%\n\n"

In [67]:
# Gridsearch for randomforest
from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(RandomForestClassifier(n_estimators=40, max_features=None, max_depth=None, min_samples_leaf=5), {
    'min_samples_split' : [3,5]
}, cv=3, return_train_score=False)

## Then train the model using GridSearchCV
clf.fit(X_train9, y_train2.status_group_2)

GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=5,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=40, n_jobs=None,
                                              oob_score=False,
                                              random_s

In [68]:
# Import results into a dataframe for easy viewing
df = pd.DataFrame(clf.cv_results_)

# The Results will be pre-ordered and ranked from best to worst 
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,6.993449,0.033383,0.183665,0.004787,3,{'min_samples_split': 3},0.777496,0.775881,0.773811,0.775729,0.001508,1
1,6.764331,0.060272,0.182666,0.009030,5,{'min_samples_split': 5},0.778065,0.775726,0.771276,0.775022,0.002816,2


In [69]:
clf.best_params_

{'min_samples_split': 3}

In [70]:
'''



X TEST SET



'''

'\n\n\n\nX TEST SET\n\n\n\n'

In [71]:



dummies_ext = pd.get_dummies(X_test1.extraction_type_class2)
dummies_mgt = pd.get_dummies(X_test1.management2)
dummies_pay = pd.get_dummies(X_test1.payment_type2)
dummies_src = pd.get_dummies(X_test1.source2)
dummies_wtr = pd.get_dummies(X_test1.waterpoint_type2)
#
dummies_qtg = pd.get_dummies(X_test1.quantity_group)
dummies_qlg = pd.get_dummies(X_test1.quality_group)

# Dummy Trap: drop all the 'other' columns
dummies_ext2 = dummies_ext.drop(['other_ext'], axis='columns')
dummies_mgt2 = dummies_mgt.drop(['other_mgt'], axis='columns')
dummies_pay2 = dummies_pay.drop(['other_pay'], axis='columns')
dummies_src2 = dummies_src.drop(['other_src'], axis='columns')
dummies_wtr2 = dummies_wtr.drop(['other_wtr'], axis='columns')
#
dummies_qtg2 = dummies_qtg.drop(['unknown'], axis='columns')
dummies_qlg2 = dummies_qlg.drop(['unknown'], axis='columns')

# Combine
X_test2 = pd.concat([X_test1,dummies_ext2], axis='columns')
X_test3 = pd.concat([X_test2,dummies_mgt2], axis='columns')
X_test4 = pd.concat([X_test3,dummies_pay2], axis='columns')
X_test5 = pd.concat([X_test4,dummies_src2], axis='columns')
X_test6 = pd.concat([X_test5,dummies_wtr2], axis='columns')
#
X_test7 = pd.concat([X_test6,dummies_qtg2], axis='columns')
X_test8 = pd.concat([X_test7,dummies_qlg2], axis='columns')

# Drop varchar labels (for X and y)
X_test9 = X_test8.drop(['ID','extraction_type_class2','management2','payment_type2','source2','waterpoint_type2','quantity_group','quality_group'], axis='columns')

In [72]:

#### USE GRIDSEARCHSV best results for fitting the model (by defualt it chooses the best parameters)
y_predicted = clf.predict(X_test9)

In [73]:
result = pd.DataFrame(y_predicted)

In [74]:
result.head()

,0
0,2
1,0
2,2
3,0
4,2


In [75]:
result2 = pd.concat([result,X_test1.ID], axis='columns')
result2.head()

,0,ID
0,2,10
1,0,13
2,2,14
3,0,29
4,2,32


In [55]:
#### output for 1 column target
result2.to_sql(
    name='watertable_y_test_result2', 
    con=sqlcon,
    index=False,
    schema='DRIVENDATA.dbo',
    if_exists='replace' 
)

NameError: name 'result2' is not defined